In [1]:
import os
import sys
from tqdm import tqdm
from PIL import Image
import numpy as np

sys.path.extend(['..'])

from utils.config import process_config

import tensorflow as tf
from tensorflow.layers import (conv2d, max_pooling2d, average_pooling2d, batch_normalization, dropout, dense)
from tensorflow.nn import (relu, sigmoid, softmax, leaky_relu)

from sklearn.utils import shuffle

In [2]:
DATA = '../data/data_clean/'
CONF = '../configs/roman.json'

In [3]:
def normalize(image):
    return (image - image.min()) / (image.max() - image.min())

def shuffle_sim(a, b):
    assert a.shape[0] == a.shape[0], 'Shapes must be equal'
    
    ind = np.arange(a.shape[0])
    np.random.shuffle(ind)
    return a[ind], b[ind]

In [4]:
def read_train_test(path_to_data):
    data = {}
    for dset in ['train', 'test']:
        path_ = os.path.join(path_to_data, dset)
        X, Y = [], []
        classes = [d for d in os.listdir(path_) if os.path.isdir(os.path.join(path_, d))]
        classes.sort()
        
        for cl in classes:
            y = np.zeros((1, 8), dtype=np.int32)
            y[0, int(cl) - 1] = 1
            
            cl_path = os.path.join(path_, cl)
            filenames = [os.path.join(cl_path, pict) for pict in os.listdir(cl_path) if pict.endswith('.jpg')]
            
            for im in filenames:
                image = np.asarray(Image.open(im), dtype=np.float32)
                X.append(normalize(image).reshape((1, image.shape[0], image.shape[1], image.shape[2])))
                Y.append(y)
        
        a, b = shuffle_sim(np.concatenate(X), np.concatenate(Y))
        data[dset] = ([a, b])
    return data

In [5]:
class Model():
    
    def __init__(self, config, sess_cf, learning_rate):
        self.lr = learning_rate
        self.sess = tf.Session(config=sess_cf)

        self.x = tf.placeholder(dtype=tf.float32, shape=(None, config.image_size, config.image_size, 3))
        self.y = tf.placeholder(dtype=tf.int32, shape=(None, 8))
        self.training = tf.placeholder(dtype=tf.bool, shape=())

        global_step = tf.Variable(1, name='global_step', trainable=False, dtype=tf.int32)
        self.step = tf.assign(global_step, global_step + 1)
        
        self.model()
        
        self.summ_writer = tf.summary.FileWriter(config.summary_dir, graph=self.sess.graph)
        self.sess.run(tf.global_variables_initializer())
    
    def block(self, inp,
              ch,
              num,
              c_ker=[(3, 3), (3, 3)],
              c_str=[(1, 1), (1, 1)],
              act=relu,
              mp_ker=(2, 2),
              mp_str=(2, 2),
              mode='conc'):
    
        with tf.variable_scope('block_' + str(num), reuse=tf.AUTO_REUSE):
            conv = conv2d(inp, ch, c_ker[0], strides=c_str[0])
            bn = batch_normalization(conv)
            out = act(bn)
            out = dropout(out, 0.2)
            tf.summary.histogram('conv1', conv)
            print(out.shape)
            
            conv = conv2d(out, ch, c_ker[1], strides=c_str[1])
            bn = batch_normalization(conv)
            out = act(bn)
            tf.summary.histogram('conv2', conv)
            print(out.shape)
            
            mp = max_pooling2d(out, mp_ker, strides=mp_str)
            if mode == 'mp':
                out = mp
            elif mode == 'conc':
                out = tf.concat([mp, average_pooling2d(out, mp_ker, mp_str)], -1)
            else:
                raise ValueError('Unknown mode.')
            
            print(out.shape)
        return out
    
    def model(self):
        with tf.name_scope('layers'):
            out = self.block(self.x, 16, 1, mode='conc')
            out = self.block(out, 32, 2, mode='conc')
            out = self.block(out, 64, 3, mode='conc')
            out = self.block(out, 256, 4, c_str=[(1, 1), (2, 2)], mode='mp')
            
            dim = np.prod(out.shape[1:])
            out = tf.reshape(out, [-1, dim])
            print(out.shape)
            
            dense_l = dense(out, 128)
            tf.summary.histogram('dense2', dense_l)
            out = batch_normalization(dense_l)
            out = leaky_relu(out, alpha=0.01)
            out = dropout(out, rate=0.7, training=self.training)
            print(out.shape)

            self.predictions = dense(out, 8, activation=softmax)
            tf.summary.histogram('pred', self.predictions)

        with tf.name_scope('metrics'):    
            amax_labels = tf.argmax(self.y, 1)
            amax_pred   = tf.argmax(self.predictions, 1)

            self.loss = tf.losses.softmax_cross_entropy(self.y, self.predictions)        
            self.acc = tf.reduce_mean(tf.cast(tf.equal(amax_labels, amax_pred), dtype=tf.float32))

            self.optimizer = tf.train.AdamOptimizer(self.lr).minimize(self.loss)

            tf.summary.scalar('loss', self.loss)
            tf.summary.scalar('accuracy', self.acc)

        self.summary = tf.summary.merge_all()
        
    def train(self, dat, dat_v, epochs):
        for epoch in range(epochs):
            loss, acc, _, summary, step = self.sess.run([
                self.loss, self.acc, self.optimizer, self.summary, self.step
            ],
                                                feed_dict={
                                                    self.x: dat[0],
                                                    self.y: dat[1],
                                                    self.training: True
                                                })

            self.summ_writer.add_summary(summary, step)
            print('EP: {:3d}\tLOSS: {:.10f}\tACC: {:.10f}'.format(
                epoch, loss, acc))

            if epoch % 10 == 0 and epoch != 0:
                self.test(dat_v)
                
    def test(self, dat):
        loss, acc = self.sess.run([self.loss, self.acc],
                                         feed_dict={self.x: dat[0],
                                                    self.y: dat[1],
                                                    self.training: False})

        print('\tVALIDATION\tLOSS: {:.10f}\tACC: {:.10f}'.format(loss, acc))
    
    def close(self):
        self.sess.close()
        tf.reset_default_graph()
        

In [12]:
m.close()

In [6]:
config_tf = tf.ConfigProto(allow_soft_placement=True)
config_tf.gpu_options.allow_growth = True
config_tf.gpu_options.per_process_gpu_memory_fraction = 0.95

In [12]:
config = process_config(CONF)
config['exp_name'] = '4b_clean_data'
config['summary_dir'] = '../experiments/' + config['exp_name'] + '/summary'

In [13]:
config = process_config(CONF)
im_size = config.image_size
# im_size = config['image_size'] is also works

128

In [8]:
m = Model(config, config_tf, 5e-4)

(?, 126, 126, 16)
(?, 124, 124, 16)
(?, 62, 62, 32)
(?, 60, 60, 32)
(?, 58, 58, 32)
(?, 29, 29, 64)
(?, 27, 27, 64)
(?, 25, 25, 64)
(?, 12, 12, 128)
(?, 10, 10, 256)
(?, 4, 4, 256)
(?, 2, 2, 256)
(?, 1024)
(?, 128)


In [9]:
dat = read_train_test(DATA)

In [10]:
m.train(dat['train'], dat['test'], 400)

EP:   0	LOSS: 2.0792574883	ACC: 0.1280276775
EP:   1	LOSS: 2.0785179138	ACC: 0.1314878911
EP:   2	LOSS: 2.0783586502	ACC: 0.1332179904
EP:   3	LOSS: 2.0800726414	ACC: 0.1435986161
EP:   4	LOSS: 2.0764451027	ACC: 0.1418685168
EP:   5	LOSS: 2.0783786774	ACC: 0.1418685168
EP:   6	LOSS: 2.0789024830	ACC: 0.1314878911
EP:   7	LOSS: 2.0771849155	ACC: 0.1522491276
EP:   8	LOSS: 2.0755751133	ACC: 0.1349480897
EP:   9	LOSS: 2.0766043663	ACC: 0.1211072654
EP:  10	LOSS: 2.0753393173	ACC: 0.1349480897
	VALIDATION	LOSS: 2.0752797127	ACC: 0.1320754737
EP:  11	LOSS: 2.0745828152	ACC: 0.1384083033
EP:  12	LOSS: 2.0780797005	ACC: 0.1435986161
EP:  13	LOSS: 2.0744574070	ACC: 0.1505190283
EP:  14	LOSS: 2.0698380470	ACC: 0.1522491276
EP:  15	LOSS: 2.0729060173	ACC: 0.1487889290
EP:  16	LOSS: 2.0708818436	ACC: 0.1435986161
EP:  17	LOSS: 2.0754349232	ACC: 0.1366782039
EP:  18	LOSS: 2.0685412884	ACC: 0.1297577918
EP:  19	LOSS: 2.0717496872	ACC: 0.1349480897
EP:  20	LOSS: 2.0705764294	ACC: 0.1332179904
	VALID

EP: 165	LOSS: 1.4252028465	ACC: 0.8460207582
EP: 166	LOSS: 1.4219763279	ACC: 0.8494809866
EP: 167	LOSS: 1.4179451466	ACC: 0.8633217812
EP: 168	LOSS: 1.4273074865	ACC: 0.8442906737
EP: 169	LOSS: 1.4329737425	ACC: 0.8460207582
EP: 170	LOSS: 1.4280627966	ACC: 0.8460207582
	VALIDATION	LOSS: 1.4023411274	ACC: 0.8726415038
EP: 171	LOSS: 1.4169806242	ACC: 0.8633217812
EP: 172	LOSS: 1.4092594385	ACC: 0.8667820096
EP: 173	LOSS: 1.4154653549	ACC: 0.8581314683
EP: 174	LOSS: 1.4157845974	ACC: 0.8546712995
EP: 175	LOSS: 1.4144563675	ACC: 0.8650519252
EP: 176	LOSS: 1.3911622763	ACC: 0.8875432611
EP: 177	LOSS: 1.4103302956	ACC: 0.8702422380
EP: 178	LOSS: 1.4154181480	ACC: 0.8564013839
EP: 179	LOSS: 1.4216790199	ACC: 0.8512110710
EP: 180	LOSS: 1.4266048670	ACC: 0.8442906737
	VALIDATION	LOSS: 1.3757122755	ACC: 0.9009433985
EP: 181	LOSS: 1.4096963406	ACC: 0.8650519252
EP: 182	LOSS: 1.4216514826	ACC: 0.8494809866
EP: 183	LOSS: 1.4067628384	ACC: 0.8650519252
EP: 184	LOSS: 1.4297170639	ACC: 0.8477508426
EP

EP: 330	LOSS: 1.3208101988	ACC: 0.9515570998
	VALIDATION	LOSS: 1.3215858936	ACC: 0.9528301954
EP: 331	LOSS: 1.3094278574	ACC: 0.9688581228
EP: 332	LOSS: 1.3151898384	ACC: 0.9619377255
EP: 333	LOSS: 1.3172676563	ACC: 0.9619377255
EP: 334	LOSS: 1.3131403923	ACC: 0.9653978944
EP: 335	LOSS: 1.3189922571	ACC: 0.9532871842
EP: 336	LOSS: 1.3248375654	ACC: 0.9480968714
EP: 337	LOSS: 1.3140869141	ACC: 0.9619377255
EP: 338	LOSS: 1.2970807552	ACC: 0.9809688330
EP: 339	LOSS: 1.3098562956	ACC: 0.9671280384
EP: 340	LOSS: 1.3134199381	ACC: 0.9619377255
	VALIDATION	LOSS: 1.3262406588	ACC: 0.9481132030
EP: 341	LOSS: 1.3114860058	ACC: 0.9636678100
EP: 342	LOSS: 1.3137609959	ACC: 0.9619377255
EP: 343	LOSS: 1.3224085569	ACC: 0.9515570998
EP: 344	LOSS: 1.3137152195	ACC: 0.9602076411
EP: 345	LOSS: 1.3158627748	ACC: 0.9619377255
EP: 346	LOSS: 1.3178564310	ACC: 0.9567474127
EP: 347	LOSS: 1.3134284019	ACC: 0.9602076411
EP: 348	LOSS: 1.3122661114	ACC: 0.9653978944
EP: 349	LOSS: 1.3130257130	ACC: 0.9619377255
EP

In [11]:
m.test(dat['test'])

	VALIDATION	LOSS: 1.3120685816	ACC: 0.9622641802
